<a href="https://colab.research.google.com/github/emiliehp/DAT300/blob/emilie/CA2_CNN_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compulsory Assignment 2: Convolutional neural networks

Please fill out the group name, number, members and optionally the name below.

**Group number**: 19\
**Group member 1**: Anne Berit Alm \
**Group member 2**: Benjamin Andersen \
**Group member 3**: Emilie Hope Petersen\
**Group name (optional)**:


# Assignment Submission
To complete this assignment answer the relevant questions in this notebook and write the code required to implement the relevant models. The assignment is submitted by handing in this notebook as an .ipynb file and as a .pdf file.

# Introduction
In this assignment, you will build a Convolutional Neural Network (CNN) to classify images of natural scenes from around the world.
Dataset: Intel-image-classification
https://www.kaggle.com/datasets/puneet6060/intel-image-classification

This Data contains around 25k images of size 150x150 distributed under 6 categories.
{'buildings' -> 0,
'forest' -> 1,
'glacier' -> 2,
'mountain' -> 3,
'sea' -> 4,
'street' -> 5 }

#### In CA2, we use only 3 classes "buildings", "forest", "sea".

This data was initially published on https://datahack.analyticsvidhya.com by Intel to host a Image classification Challenge.
## Landscape Pictures

Example image:


<center><img src="https://github.com/emiliehp/DAT300/blob/main/20497.jpg?raw=1" width="500" height="400"></center>


## Assignment structure

1. Part 0: Setup & Data
2. Part 1: Baseline CNN (Clean Data)
3. Part 2: Choose Your Robustness Challenges
4. Part 3: Results & Comparison

```

## Library imports

In [ ]:
# Open files from disk
from google.colab import drive
drive.mount('/content/drive')

!unzip -o "/content/drive/MyDrive/DAT300/CA2.zip" -d project
!ls project
import sys
sys.path.append('/content/project/CA2')


In [34]:
# Feel free to add or remove libraries as you want
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras as ks
from utilities import f1_score
import tensorflow as tf


# Part 0: Setup & Data, CNN for Landscape Picture dataset
### NOTE FOR STUDENTS:

Do not forget to change the path of root_dir below so it points to the Intel folder which contains the dataset inside your CA2 directory.
### Example:

Windows: r"C:\Users\yourname\...\CA2\Intel"

Mac/Linux: "/home/yourname/CA2/Intel"

## Loading DATASET

In [35]:
import importlib
import utilities

# Force reload the updated utilities.py
importlib.reload(utilities)

from utilities import load_intel_dataset


# Load dataset
data = load_intel_dataset(
    root_dir=r"/content/project/CA2/Intel",
    img_size=(128, 128),
    selected_classes=["buildings", "forest", "sea"],  #  only 3 classes
    verbose=1

)

X_train, y_train = data["X_train"], data["y_train"]
X_val,   y_val   = data["X_val"],   data["y_val"]
X_test,  y_test  = data["X_test"],  data["y_test"]
class_names      = data["class_names"]

print("Train:", X_train.shape, y_train.shape)
print("Val:",   X_val.shape,   y_val.shape)
print("Test:",  X_test.shape,  y_test.shape)

# Show number of images per class
def count_per_class(y, split_name):
    unique, counts = np.unique(y, return_counts=True)
    print(f"\n{split_name} set class distribution:")
    for cls, cnt in zip(unique, counts):
        print(f"  {class_names[cls]}: {cnt}")

count_per_class(y_train, "Train")
count_per_class(y_val,   "Val")
count_per_class(y_test,  "Test")


[train] Loaded 500/6736 images
[train] Loaded 1000/6736 images
[train] Loaded 1500/6736 images
[train] Loaded 2000/6736 images
[train] Loaded 2500/6736 images
[train] Loaded 3000/6736 images
[train] Loaded 3500/6736 images
[train] Loaded 4000/6736 images
[train] Loaded 4500/6736 images
[train] Loaded 5000/6736 images
[train] Loaded 5500/6736 images
[train] Loaded 6000/6736 images
[train] Loaded 6500/6736 images
[train] Loaded 6736/6736 images
[test] Loaded 500/1421 images
[test] Loaded 1000/1421 images
[test] Loaded 1421/1421 images
Final shapes → Train: (6062, 128, 128, 3), Val: (674, 128, 128, 3), Test: (1421, 128, 128, 3)
Classes: ['buildings', 'forest', 'sea']
Train: (6062, 128, 128, 3) (6062,)
Val: (674, 128, 128, 3) (674,)
Test: (1421, 128, 128, 3) (1421,)

Train set class distribution:
  buildings: 1972
  forest: 2044
  sea: 2046

Val set class distribution:
  buildings: 219
  forest: 227
  sea: 228

Test set class distribution:
  buildings: 437
  forest: 474
  sea: 510


## Preprocessing


In [36]:

# Normalizing input between [0,1]
X_train = X_train.astype("float32") / 255.0 if X_train.size else X_train
X_val   = X_val.astype("float32")   / 255.0 if X_val.size else X_val
X_test  = X_test.astype("float32")  / 255.0 if X_test.size else X_test

# Converting targets from numbers to categorical format
if y_train is not None and y_train.size:
    num_classes = len(np.unique(y_train))
    y_train = ks.utils.to_categorical(y_train, num_classes)

if y_val is not None and y_val.size:
    y_val = ks.utils.to_categorical(y_val, num_classes)

if y_test is not None and y_test.size:
    y_test = ks.utils.to_categorical(y_test, num_classes)

print("X_train shape:", X_train.shape)
print("X_val shape:",   X_val.shape)
print("X_test shape:",  X_test.shape)



X_train shape: (6062, 128, 128, 3)
X_val shape: (674, 128, 128, 3)
X_test shape: (1421, 128, 128, 3)


## Part 1: Build a CNN network with the LeNet5 architecture

##### Implement LeNet5 architecture for Landscape Pictures (RGB):

--------------------------
The LeNet architecture takes a 32×32×C image as input, where C is the number of color channels. You may use resize_dataset() and f1_score() from utilities.py¶

Input & resizing: Resize all images to 32×32 and use C = 3 channels (RGB).
If you choose a different input size, update the intermediate shapes accordingly, but keep the LeNet-5 pattern (Conv → Pool → Conv → Pool → FC → FC → FC).

**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU.

**MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU.

**MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected (output):** **`num_classes`**. **Activation:** softmax

--------------------------


##### Compile the network with the
* `tf.keras.losses.CategoricalCrossentropy` loss function
* the `adam` optimizer
* with the `accuracy` metric and (your own implementation of the) F1-score metric.

In [38]:
# Building CNN network with LeNet5 arch
# First resizing
import tensorflow as tf
from utilities import resize_dataset
X_train, y_train = resize_dataset(X_train, (32, 32))

# Take input 32x32, C = 3
def lenet5(input_shape=(32, 32, 3), num_classes=3):
  model = ks.models.Sequential([
      ks.layers.Conv2D(6, (5,5), activation="relu", padding="valid", input_shape=input_shape),
      ks.layers.MaxPooling2D((2,2)),
      ks.layers.Conv2D(16, (5,5), activation="relu", padding="valid"),
      ks.layers.MaxPooling2D((2,2)),
      ks.layers.Flatten(),
      ks.layers.Dense(120, activation="relu"),
      ks.layers.Dense(84, activation="relu"),
      ks.layers.Dense(num_classes, activation="softmax")
  ])
  return model

# Compile network and check the output layers shapes
num_classes = len(class_names)
model = lenet5((32,32,3), num_classes)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', f1_score]
)

model.summary()


NameError: name 'tf' is not defined

### Task 1.1.2 Train network

Train the network with a
* batch size of 64 samples
* for 20 epochs


In [ ]:
# Train network
start_time = time.time()
history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=20,
    validation_data=(X_val, y_val)
)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")



### Task 1.1.3
Experiment with different architectures of your choice. Vary the number of filters, try different kernel sizes, add more layers, dropout, early stopping, and modify the fully connected layers. Report the best performance you achieve.

## Task 1.2 Evaluaiton
### Task 1.2.1 Plot training history
- Plot the training/validation accuracy and loss curves (plot_training_history() is in utilities.py).
- Report the final validation accuracy (f1_score() is in utilities.py).

### Task 1.2.2 Evaluate on the test dataset
- Report test accuracy (and F1 score (f1_score() is in utilities.py)).

### Task 1.2.3 Create a confusion matrix for both training and testing data
- Visualize confusion matrices.
- Do the test data and train data predict the same items wrong?

## Part 2: Robustness (choose two or more), You may use add_gaussian_noise() and add_motion_blur() from utilities.py

### Task 2.1 Data Augmentation (training-time)
- Implement an augmentation pipeline.
- Train a model with augmentation and compare against the baseline from Part 1.

### Task 2.2 Noise Robustness (test-time corruptions)
- Create corrupted versions of the test images and evaluate the baseline model:
  - Gaussian noise
  - Motion blur
- Report accuracy on the clean test set vs. each corrupted test set.

### Task 2.3  Discussion (no coding)
- Which noise types affect the model most?
- What techniques could improve robustness (data augmentation, adversarial training, denoising pre-processing, larger models)?

# Part 3: Results & Comparison
- Summarize results (table/plot):
  - Baseline (clean) vs. Augmented (if attempted)
  - Clean vs. each corrupted test set (Task 2.2)
- Brief discussion:
  - Which corruptions hurt most, and why?
  - Did augmentation help? Which transforms mattered?
  - What would you try next (e.g., stronger augmentation, adversarial training, denoising pre-processing, larger models, early stopping, ensembling)?